In [1]:
 # -*- coding: utf-8 -*-
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
from PIL import Image
import cv2
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
plt.rcParams["figure.figsize"] = (15,15)

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
config = tf.ConfigProto(gpu_options=gpu_options)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

Instructions for updating:
non-resource variables are not supported in the long term
Num GPUs Available:  1


In [2]:
height = 32
width = 32
channels = 3
n_inputs = height * width * channels

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps =64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"


conv3_fmaps = 128
conv3_ksize = 4
conv3_stride = 1
conv3_pad = "SAME"
pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

tf.reset_default_graph()




In [3]:

cifar_file = "/home/vaughan/AdvMl/Datasets AdvMl/cifar-10-batches-py/data_batch_"
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

cf = unpickle(cifar_file + "1")
m = 32
n = 32
offset = 0

X_train = np.empty([50000,32,32,3])
y_train = np.empty([50000])



for i in range(5):
    cf = unpickle(cifar_file + str(i+1))
    instances = cf[b'data'].reshape(10000, 3, m, n).transpose(0,2,3,1).astype("uint8")
    labels = cf[b'labels']

    for i,val in enumerate(instances):
        X_train[offset+i,:,:,:] = val
    y_train[offset : offset+len(labels)] = labels

    offset += len(labels)

cf = unpickle("/home/vaughan/AdvMl/Datasets AdvMl/cifar-10-batches-py/test_batch")
X_test = cf[b'data'].reshape(10000, 3, m, n).transpose(0,2,3,1).astype("uint8")
y_test = cf[b'labels']
labelmap = {
    0:"airplane",
    1:"automobile",
    2:"bird",
    3:"cat",
    4:"deer",
    5:"dog",
    6:"frog",
    7:"horse",
    8:"ship",
    9:"truck"
}

In [4]:
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, height,width,channels], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.selu, name="conv1")

conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.tanh, name="conv2")




Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [5]:
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 8  *8])



In [6]:
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.selu, name="fc1")



Instructions for updating:
Use keras.layers.Dense instead.


In [7]:
with tf.name_scope("output"):
    hidden_1 = tf.layers.dense(fc1,500,activation=tf.nn.tanh, name="hidden_1")
    hidden_2 = tf.layers.dense(hidden_1,400,activation=tf.nn.tanh, name="hidden_2")
    hidden_3 = tf.layers.dense(hidden_2,200,activation=tf.nn.tanh, name="hidden_3")
    hidden_4 = tf.layers.dense(hidden_3,200,activation=tf.nn.tanh, name="hidden_4")
    logits = tf.layers.dense(hidden_4, n_outputs, name="output")
    print(logits.get_shape())
    Y_proba = tf.nn.softmax(logits, name="Y_proba")



(?, 10)


In [8]:
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)



In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))



In [10]:
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver(tf.global_variables())


In [11]:
n_epochs =10
batch_size = 350
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
        
print(X_train.shape)
print("conv1: ",conv1.get_shape())
print("conv2: ",conv2.get_shape())
print("pool3: ",pool3.get_shape())
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train,"Test accuracy:", acc_test)
        saver.save(sess,"model")

with tf.Session() as sess:
    # Restore the model
    tf_saver = tf.train.Saver()
    tf_saver.restore(sess,"model")
    
    im = cv2.imread("../Python/Project/testImgs205/my3.png")
    im = im.reshape(1,32,32,3)
    print(tf.argmax(Y_proba.eval(feed_dict={X:im})[0]).eval())

(50000, 32, 32, 3)
conv1:  (?, 32, 32, 32)
conv2:  (?, 16, 16, 64)
pool3:  (?, 8, 8, 64)
0 Train accuracy: 0.4375 Test accuracy: 0.4391
1 Train accuracy: 0.55965906 Test accuracy: 0.5119
2 Train accuracy: 0.61647725 Test accuracy: 0.5632
3 Train accuracy: 0.6363636 Test accuracy: 0.5539
4 Train accuracy: 0.62784094 Test accuracy: 0.6033
5 Train accuracy: 0.64204544 Test accuracy: 0.6107
6 Train accuracy: 0.67045456 Test accuracy: 0.6235
7 Train accuracy: 0.71590906 Test accuracy: 0.6334
8 Train accuracy: 0.68465906 Test accuracy: 0.6448
9 Train accuracy: 0.7755682 Test accuracy: 0.6597
INFO:tensorflow:Restoring parameters from model
0
